# Application for estimating the grades of portuguese students

## 1. Import all needed libraries

In [1]:
import os

import tkinter as tk
import pandas as pd

from typing import Any
from tkinter import ttk
from tkinter import filedialog as fd

from pgmpy.models import BayesianNetwork

from app.preprocessing import TunedPreprocessing, Preprocessing
from app.network import Network, NetworkG1G2, NetworkG1OrG2

## 2. Settings for the application

In [2]:
TRAIN_PATH = "../data/student-por_2.csv"
DELIMITER = ";"
NUMBER_CORES = 10 # !IMPORTANT: Set this to the number of cores of your machine for parallelization. It is recommended to use not all cores.

## 3. Define Application

In [3]:
desc_1 = "Please select a file to predict the final grade of a student *OR* define the input values manually."

school_settings = {"label": "School", "map_values": ["GP", "MS"], "text": ["Gabriel Pereira", "Mousinho da Silveira"]}
sex_settings = {"label": "Gender", "map_values": ["F", "M"], "text": ["Female", "Male"]}
age_settings = {"label": "Age", "map_values": [i for i in range(15, 23, 1)], "text": [str(i) for i in range(15, 23, 1)]}
address_settings = {"label": "Address", "map_values": ["U", "R"], "text": ["Urban", "Rural"]}
family_size_settings = {"label": "Family Size", "map_values": ["LE3", "GT3"], "text": ["Less than or equal to 3", "Greater than 3"]}
Pstatus_settings = {"label": "Parent's Cohabitation Status", "map_values": ["T", "A"], "text": ["Living together", "Apart"]}
Fedu_settings = {"label": "Mother's Education", "map_values": [i for i in range(0, 5, 1)], "text": ["None", "Primary Education (4th grade)", "5th to 9th grade", "Secondary Education", "Higher Education"]}
Medu_settings = {"label": "Father's Education", "map_values": [i for i in range(0, 5, 1)], "text": ["None", "Primary Education (4th grade)", "5th to 9th grade", "Secondary Education", "Higher Education"]}
Fjob_settings = {"label": "Mother's Job", "map_values": ["teacher", "health", "services", "at_home", "other"], "text": ["Teacher", "Healthcare", "Civil Services", "At Home", "Other"]}
Mjob_settings = {"label": "Father's Job", "map_values": ["teacher", "health", "services", "at_home", "other"], "text": ["Teacher", "Healthcare", "Civil Services", "At Home", "Other"]}
reason_settings = {"label": "Reason for Choosing this School", "map_values": ["home", "reputation", "course", "other"], "text": ["Close to Home", "School Reputation", "Course", "Other"]}
guardian_settings = {"label": "Guardian", "map_values": ["mother", "father", "other"], "text": ["Mother", "Father", "Other"]}
traveltime_settings = {"label": "Travel Time", "map_values": [i for i in range(1, 5, 1)], "text": ["Less than 15 min", "15 to 30 min", "30 min to 1 hour", "More than 1 hour"]}
studytime_settings = {"label": "Weekly Study Time", "map_values": [i for i in range(1, 5, 1)], "text": ["Less than 2 hours", "2 to 5 hours", "5 to 10 hours", "More than 10 hours"]}
failures_settings = {"label": "Number of Past Class Failures", "map_values": [i for i in range(0, 4, 1)], "text": ["None", "1", "2", "3 or more"]}
schoolsup_settings = {"label": "Extra Educational Support", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
famsup_settings = {"label": "Family Educational Support", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
paid_settings = {"label": "Extra Paid Classes within the Course Subject", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
activities_settings = {"label": "Extra Curricular Activities", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
nursery_settings = {"label": "Attended Nursery School", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
higher_settings = {"label": "Wants to Take Higher Education", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
internet_settings = {"label": "Internet Access at Home", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
romantic_settings = {"label": "In a Romantic Relationship", "map_values": ["yes", "no"], "text": ["Yes", "No"]}
famrel_settings = {"label": "Quality of Family Relationships", "map_values": [i for i in range(1, 6, 1)], "text": ["Very Bad", "Bad", "Neutral", "Good", "Very Good"]}
free_time_settings = {"label": "Free Time after School", "map_values": [i for i in range(1, 6, 1)], "text": ["Very Low", "Low", "Neutral", "High", "Very High"]}
goout_settings = {"label": "Going Out with Friends", "map_values": [i for i in range(1, 6, 1)], "text": ["Very Low", "Low", "Neutral", "High", "Very High"]}
Dalc_settings = {"label": "Workday Alcohol Consumption", "map_values": [i for i in range(1, 6, 1)], "text": ["Very Low", "Low", "Neutral", "High", "Very High"]}
Walc_settings = {"label": "Weekend Alcohol Consumption", "map_values": [i for i in range(1, 6, 1)], "text": ["Very Low", "Low", "Neutral", "High", "Very High"]}
health_settings = {"label": "Current Health Status", "map_values": [i for i in range(1, 6, 1)], "text": ["Very Bad", "Bad", "Neutral", "Good", "Very Good"]}
absences_settings = {"label": "Number of School Absences", "map_values": [i for i in range(0, 93, 1)], "text": [str(i) for i in range(0, 93, 1)]}


In [4]:

class App(tk.Tk):
    PAD_X = 32
    PAD_Y = 8
    X_MAX = 1200
    Y_MAX = 700
    WRAP_LENGTH = X_MAX - (PAD_X * 2)
    
    def __init__(self):
        super().__init__()
        
        self._original_data: pd.DataFrame | None = None
        
        self._path: str = ""
        self._csv_data: pd.DataFrame | None = None
        self._network_g1_g2: BayesianNetwork | None = None
        self._networl_g1_or_g2: BayesianNetwork | None = None
        
        self.build_ui()
        self.create_networks()
    
    def build_ui(self) -> None:
        self.title("Student Grade Prediction")
        self.geometry(f"{self.X_MAX}x{self.Y_MAX}")
        self.resizable(False, False)
        
        self.header = ttk.Label(self, text="Student Grade predictor", font=("Arial", 16, "bold"))
        self.header.pack(padx=self.PAD_X, pady=self.PAD_Y)
        self.dsc_1 = ttk.Label(self, text=desc_1, wraplength=self.WRAP_LENGTH)
        self.dsc_1.pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
        # Open CSV file
        self.open_csv_btn = ttk.Button(self, text="Load Students CSV file", command=self.select_csv)
        self.open_csv_btn.pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
        self.reset_path_btn = ttk.Button(self, text="Reset Path", command=self.reset_path, state="disabled")
        self.reset_path_btn.pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
        
        # Path label
        self.path_lbl = ttk.Label(self, text=f"Path: {self._path if self._path else 'Not selected'}", wraplength=512)
        self.path_lbl.pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
        # Divider
        hr = ttk.Separator(self, orient="horizontal").pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
        # Form
        self.build_form()
        
        # Predict button
        self.predict_btn = ttk.Button(self, text="Predict", command=self.predict)
        self.predict_btn.pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
        # Divider
        hr2 = ttk.Separator(self, orient="horizontal").pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
        # Result
        self.result_lbl = ttk.Label(self, text="", font=("Arial", 16, "bold"))
        self.result_lbl.pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
    def build_form(self) -> None:
        self.form = ttk.Frame(self)
        left_container = ttk.Frame(self.form)
        left_container.columnconfigure(0, weight=1)
        left_container.columnconfigure(1, weight=4)

        # School
        ttk.Label(left_container, text=school_settings["label"]).grid(padx=4, pady=2, row=0, column=0, sticky="w")
        self.cmbx_school = ttk.Combobox(left_container, values=school_settings["text"], state="readonly")
        self.cmbx_school.grid(row=0, column=1, sticky="we")
        
        # Sex
        ttk.Label(left_container, text=sex_settings["label"]).grid(padx=4, pady=2,row=1, column=0, sticky="w")
        self.cmbx_sex = ttk.Combobox(left_container, values=sex_settings["text"], state="readonly")
        self.cmbx_sex.grid(row=1, column=1, sticky="we")
        
        # Age
        ttk.Label(left_container, text=age_settings["label"]).grid(padx=4, pady=2,row=2, column=0, sticky="w")
        self.cmbx_age = ttk.Combobox(left_container, values=age_settings["text"], state="readonly")
        self.cmbx_age.grid(row=2, column=1, sticky="we")
        
        # Address
        ttk.Label(left_container, text=address_settings["label"]).grid(padx=4, pady=2,row=3, column=0, sticky="w")
        self.cmbx_address = ttk.Combobox(left_container, values=address_settings["text"], state="readonly")
        self.cmbx_address.grid(row=3, column=1, sticky="we")
        
        # Family size
        ttk.Label(left_container, text=family_size_settings["label"]).grid(padx=4, pady=2,row=4, column=0, sticky="w")
        self.cmbx_family_size = ttk.Combobox(left_container, values=family_size_settings["text"], state="readonly")
        self.cmbx_family_size.grid(row=4, column=1, sticky="we")
        
        # Pstatus
        ttk.Label(left_container, text=Pstatus_settings["label"]).grid(padx=4, pady=2,row=5, column=0, sticky="w")
        self.cmbx_Pstatus = ttk.Combobox(left_container, values=Pstatus_settings["text"], state="readonly")
        self.cmbx_Pstatus.grid(row=5, column=1, sticky="we")
        
        # Medu
        ttk.Label(left_container, text=Medu_settings["label"]).grid(padx=4, pady=2,row=6, column=0, sticky="w")
        self.cmbx_medu = ttk.Combobox(left_container, values=Medu_settings["text"], state="readonly")
        self.cmbx_medu.grid(row=6, column=1, sticky="we")
        
        # Fedu
        ttk.Label(left_container, text=Fedu_settings["label"]).grid(padx=4, pady=2,row=7, column=0, sticky="w")
        self.cmbx_fedu = ttk.Combobox(left_container, values=Fedu_settings["text"], state="readonly")
        self.cmbx_fedu.grid(row=7, column=1, sticky="we")
        
        # Mjob
        ttk.Label(left_container, text=Mjob_settings["label"]).grid(padx=4, pady=2,row=8, column=0, sticky="w")
        self.cmbx_mjob = ttk.Combobox(left_container, values=Mjob_settings["text"], state="readonly")
        self.cmbx_mjob.grid(row=8, column=1, sticky="we")
        
        # Fjob
        ttk.Label(left_container, text=Fjob_settings["label"]).grid(padx=4, pady=2,row=9, column=0, sticky="w")
        self.cmbx_fjob = ttk.Combobox(left_container, values=Fjob_settings["text"], state="readonly")
        self.cmbx_fjob.grid(row=9, column=1, sticky="we")
        
        # Reason
        ttk.Label(left_container, text=reason_settings["label"]).grid(padx=4, pady=2,row=10, column=0, sticky="w")
        self.cmbx_reason = ttk.Combobox(left_container, values=reason_settings["text"], state="readonly")
        self.cmbx_reason.grid(row=10, column=1, sticky="we")
        
        # Guardian
        ttk.Label(left_container, text=guardian_settings["label"]).grid(padx=4, pady=2,row=11, column=0, sticky="w")
        self.cmbx_guardian = ttk.Combobox(left_container, values=guardian_settings["text"], state="readonly")
        self.cmbx_guardian.grid(row=11, column=1, sticky="we")
        
        # Traveltime
        ttk.Label(left_container, text=traveltime_settings["label"]).grid(padx=4, pady=2,row=12, column=0, sticky="w")
        self.cmbx_traveltime = ttk.Combobox(left_container, values=traveltime_settings["text"], state="readonly")
        self.cmbx_traveltime.grid(row=12, column=1, sticky="we")
        
        # Studytime
        ttk.Label(left_container, text=studytime_settings["label"]).grid(padx=4, pady=2,row=13, column=0, sticky="w")
        self.cmbx_studytime = ttk.Combobox(left_container, values=studytime_settings["text"], state="readonly")
        self.cmbx_studytime.grid(row=13, column=1, sticky="we")
        
        # Absences
        ttk.Label(left_container, text=absences_settings["label"]).grid(padx=4, pady=2,row=14, column=0, sticky="w")
        self.cmbx_absences = ttk.Combobox(left_container, values=absences_settings["text"], state="readonly")
        self.cmbx_absences.grid(row=14, column=1, sticky="we")
        
        left_container.grid(padx=8, pady=8, row=0, column=0, sticky="w")
        
        ###### Right container ######
        right_container = ttk.Frame(self.form)
        right_container.columnconfigure(0, weight=1)
        right_container.columnconfigure(1, weight=4)
        # Failures
        ttk.Label(right_container, text=failures_settings["label"]).grid(padx=4, pady=2,row=0, column=0, sticky="w")
        self.cmbx_failures = ttk.Combobox(right_container, values=failures_settings["text"], state="readonly")
        self.cmbx_failures.grid(row=0, column=1, sticky="we")
        
        # Schoolsup
        ttk.Label(right_container, text=schoolsup_settings["label"]).grid(padx=4, pady=2,row=1, column=0, sticky="w")
        self.cmbx_schoolsup = ttk.Combobox(right_container, values=schoolsup_settings["text"], state="readonly")
        self.cmbx_schoolsup.grid(row=1, column=1, sticky="we")
        
        # Famsup
        ttk.Label(right_container, text=famsup_settings["label"]).grid(padx=4, pady=2,row=2, column=0, sticky="w")
        self.cmbx_famsup = ttk.Combobox(right_container, values=famsup_settings["text"], state="readonly")
        self.cmbx_famsup.grid(row=2, column=1, sticky="we")
        
        # Paid
        ttk.Label(right_container, text=paid_settings["label"]).grid(padx=4, pady=2,row=3, column=0, sticky="w")
        self.cmbx_paid = ttk.Combobox(right_container, values=paid_settings["text"], state="readonly")
        self.cmbx_paid.grid(row=3, column=1, sticky="we")
        
        # Activities
        ttk.Label(right_container, text=activities_settings["label"]).grid(padx=4, pady=2,row=4, column=0, sticky="w")
        self.cmbx_activities = ttk.Combobox(right_container, values=activities_settings["text"], state="readonly")
        self.cmbx_activities.grid(row=4, column=1, sticky="we")
        
        # Nursery
        ttk.Label(right_container, text=nursery_settings["label"]).grid(padx=4, pady=2,row=5, column=0, sticky="w")
        self.cmbx_nursery = ttk.Combobox(right_container, values=nursery_settings["text"], state="readonly")
        self.cmbx_nursery.grid(row=5, column=1, sticky="we")
        
        # Higher
        ttk.Label(right_container, text=higher_settings["label"]).grid(padx=4, pady=2,row=6, column=0, sticky="w")
        self.cmbx_higher = ttk.Combobox(right_container, values=higher_settings["text"], state="readonly")
        self.cmbx_higher.grid(row=6, column=1, sticky="we")
        
        # Internet
        ttk.Label(right_container, text=internet_settings["label"]).grid(padx=4, pady=2,row=7, column=0, sticky="w")
        self.cmbx_internet = ttk.Combobox(right_container, values=internet_settings["text"], state="readonly")
        self.cmbx_internet.grid(row=7, column=1, sticky="we")
        
        # Romantic
        ttk.Label(right_container, text=romantic_settings["label"]).grid(padx=4, pady=2,row=8, column=0, sticky="w")
        self.cmbx_romantic = ttk.Combobox(right_container, values=romantic_settings["text"], state="readonly")
        self.cmbx_romantic.grid(row=8, column=1, sticky="we")
        
        # Family Relationship
        ttk.Label(right_container, text=famrel_settings["label"]).grid(padx=4, pady=2,row=9, column=0, sticky="w")
        self.cmbx_family_rel = ttk.Combobox(right_container, values=famrel_settings["text"], state="readonly")
        self.cmbx_family_rel.grid(row=9, column=1, sticky="we")
        
        # Free time
        ttk.Label(right_container, text=free_time_settings["label"]).grid(padx=4, pady=2,row=10, column=0, sticky="w")
        self.cmbx_free_time = ttk.Combobox(right_container, values=free_time_settings["text"], state="readonly")
        self.cmbx_free_time.grid(row=10, column=1, sticky="we")
        
        # Go out
        ttk.Label(right_container, text=goout_settings["label"]).grid(padx=4, pady=2,row=11, column=0, sticky="w")
        self.cmbx_go_out = ttk.Combobox(right_container, values=goout_settings["text"], state="readonly")
        self.cmbx_go_out.grid(row=11, column=1, sticky="we")
        
        # Dalc
        ttk.Label(right_container, text=Dalc_settings["label"]).grid(padx=4, pady=2,row=12, column=0, sticky="w")
        self.cmbx_dalc = ttk.Combobox(right_container, values=Dalc_settings["text"], state="readonly")
        self.cmbx_dalc.grid(row=12, column=1, sticky="we")
        
        # Walc
        ttk.Label(right_container, text=Walc_settings["label"]).grid(padx=4, pady=2,row=13, column=0, sticky="w")
        self.cmbx_walc = ttk.Combobox(right_container, values=Walc_settings["text"], state="readonly")
        self.cmbx_walc.grid(row=13, column=1, sticky="we")
        
        # Health
        ttk.Label(right_container, text=health_settings["label"]).grid(padx=4, pady=2,row=14, column=0, sticky="w")
        self.cmbx_health = ttk.Combobox(right_container, values=health_settings["text"], state="readonly")
        self.cmbx_health.grid(row=14, column=1, sticky="we")
        
                
        right_container.grid(padx=8, pady=8, row=0, column=1, sticky="e")
        self.form.pack(padx=self.PAD_X, pady=self.PAD_Y, fill="x")
        
    def create_networks(self) -> None:
        # Create networks and fit them
        creator_g1_g2: NetworkG1G2 = NetworkG1G2(TRAIN_PATH, NUMBER_CORES)
        creator_g1_or_g2: NetworkG1OrG2 = NetworkG1OrG2(TRAIN_PATH, NUMBER_CORES)
        creator_g1_g2.create()
        creator_g1_or_g2.create()      
        creator_g1_g2.fit()
        creator_g1_or_g2.fit()
        
        # Get networks
        self._network_g1_g2 = creator_g1_g2.get()
        self._networl_g1_or_g2 = creator_g1_or_g2.get()
    
    def reset_path(self):
        self.set_states("readonly")
        self._path = ""
        self.reset_path_btn["state"] = "disabled"
        self.result_lbl["text"] = ""
    
    def select_csv(self):
        accepted_formats = [("CSV files", "*.csv"), ("All files", "*.*")]
        self._path = fd.askopenfilename(title="Select CSV file", filetypes=accepted_formats, initialdir=os.pardir + "/data")
        if self._path == "":
            return
        self._original_data = pd.read_csv(self._path, sep=DELIMITER)
        preprocessing = TunedPreprocessing(self._original_data)
        preprocessing.process()
        self._csv_data = preprocessing.processed_data
        self.path_lbl["text"] = f"Path: {self._path}"
        self.reset_path_btn["state"] = "normal"
        self.set_states("disabled")
        
    def set_states(self, state: str = "disabled"):
        self.cmbx_school["state"] = state
        self.cmbx_sex["state"] = state
        self.cmbx_age["state"] = state
        self.cmbx_address["state"] = state
        self.cmbx_family_size["state"] = state
        self.cmbx_Pstatus["state"] = state
        self.cmbx_traveltime["state"] = state
        self.cmbx_absences["state"] = state
        self.cmbx_medu["state"] = state
        self.cmbx_fedu["state"] = state
        self.cmbx_mjob["state"] = state
        self.cmbx_fjob["state"] = state
        self.cmbx_reason["state"] = state
        self.cmbx_guardian["state"] = state
        self.cmbx_studytime["state"] = state
        self.cmbx_failures["state"] = state
        self.cmbx_schoolsup["state"] = state
        self.cmbx_famsup["state"] = state
        self.cmbx_paid["state"] = state
        self.cmbx_activities["state"] = state
        self.cmbx_nursery["state"] = state
        self.cmbx_higher["state"] = state
        self.cmbx_internet["state"] = state
        self.cmbx_romantic["state"] = state
        self.cmbx_family_rel["state"] = state
        self.cmbx_free_time["state"] = state
        self.cmbx_go_out["state"] = state
        self.cmbx_dalc["state"] = state
        self.cmbx_walc["state"] = state
        self.cmbx_health["state"] = state
        
    def _save_predictions_csv(self, predictions: list[int]):
        if self._path is None:
            print("No CSV file selected")
            return
        df = pd.DataFrame(predictions, columns=["Predicted Grade (G3)"])
        df = pd.concat([self._original_data, df], axis=1, join="inner")
        df.to_csv(self._path[:-4] + "-predicted.csv", index=False, sep=DELIMITER)
        
    def predict(self):
        if self._path:
            if 'G3' in self._csv_data.columns:
                print("G3 column found. No need to predict.")
                return
            
            predictions = self.select_model(self._csv_data).predict(self._csv_data, stochastic=False, n_jobs=NUMBER_CORES)
            self._save_predictions_csv(predictions)
            self.result_lbl["text"] = f"Predictions saved to {self._path[:-4]}-predicted.csv"
        else:
            data = {
                'school': [self.map_cmbox_text_values(self.cmbx_school.get(), school_settings)],
                'sex': [self.map_cmbox_text_values(self.cmbx_sex, sex_settings)],
                'age': [self.map_cmbox_text_values(self.cmbx_age.get(), age_settings)],
                'address': [self.map_cmbox_text_values(self.cmbx_address.get(), address_settings)],
                'famsize': [self.map_cmbox_text_values(self.cmbx_family_size.get(), family_size_settings)],
                'Pstatus': [self.map_cmbox_text_values(self.cmbx_Pstatus.get(), Pstatus_settings)],
                'traveltime': [self.map_cmbox_text_values(self.cmbx_traveltime.get(), traveltime_settings)],
                'absences': [self.map_cmbox_text_values(self.cmbx_absences.get(), absences_settings)],
                'Medu': [self.map_cmbox_text_values(self.cmbx_medu.get(), Medu_settings)],
                'Fedu': [self.map_cmbox_text_values(self.cmbx_fedu.get(), Fedu_settings)],
                'Mjob': [self.map_cmbox_text_values(self.cmbx_mjob.get(), Mjob_settings)],
                'Fjob': [self.map_cmbox_text_values(self.cmbx_fjob.get(), Fjob_settings)],
                'reason': [self.map_cmbox_text_values(self.cmbx_reason.get(), reason_settings)],
                'guardian': [self.map_cmbox_text_values(self.cmbx_guardian.get(), guardian_settings)],
                'studytime': [self.map_cmbox_text_values(self.cmbx_studytime.get(), studytime_settings)],
                'failures': [self.map_cmbox_text_values(self.cmbx_failures.get(), failures_settings)],
                'schoolsup': [self.map_cmbox_text_values(self.cmbx_schoolsup.get(), schoolsup_settings)],
                'famsup': [self.map_cmbox_text_values(self.cmbx_famsup.get(), famsup_settings)],
                'paid': [self.map_cmbox_text_values(self.cmbx_paid.get(), paid_settings)],
                'activities': [self.map_cmbox_text_values(self.cmbx_activities.get(), activities_settings)],
                'nursery': [self.map_cmbox_text_values(self.cmbx_nursery.get(), nursery_settings)],
                'higher': [self.map_cmbox_text_values(self.cmbx_higher.get(), higher_settings)],
                'internet': [self.map_cmbox_text_values(self.cmbx_internet.get(), internet_settings)],
                'romantic': [self.map_cmbox_text_values(self.cmbx_romantic.get(), romantic_settings)],
                'famrel': [self.map_cmbox_text_values(self.cmbx_family_rel.get(), famrel_settings)],
                'freeime': [self.map_cmbox_text_values(self.cmbx_free_time.get(), free_time_settings)],
                'goout': [self.map_cmbox_text_values(self.cmbx_go_out.get(), goout_settings)],
                'Dalc': [self.map_cmbox_text_values(self.cmbx_dalc.get(), Dalc_settings)],
                'Walc': [self.map_cmbox_text_values(self.cmbx_walc.get(), Walc_settings)],
                'health': [self.map_cmbox_text_values(self.cmbx_health.get(), health_settings)],
            }
            
            delete_keys = []
            for key in data:
                if data[key][0] is None:
                    delete_keys.append(key)
            
            for key in delete_keys:
                del data[key]
            
            if len(data) == 0:
                print("No data to predict")
                return
            
            df = pd.DataFrame(data)
            pre = TunedPreprocessing(df)
            pre.process()
            df = pre.processed_data
            prediction = self.select_model(df).predict(df, stochastic=False, n_jobs=NUMBER_CORES)
            self.result_lbl["text"] = f"Predicted grade (G3): {prediction.loc[0, 'G3']}"
            
    def select_model(self, data: pd.DataFrame) -> Network:
        if data is None:
            raise ValueError("Data is None")
        
        # Select model
        model = None
        if 'G1' not in data.columns and 'G2' not in data.columns:
            model = self._network_g1_g2
        else:
            model = self._networl_g1_or_g2
        return model
            
    def map_cmbox_text_values(self, text: str, settings: dict) -> Any:
        if not text or text not in settings["text"]:
            return None
        return settings["map_values"][settings["text"].index(text)]

## Start App

In [5]:
if __name__ == "__main__":
    app = App()
    app.mainloop()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/599 [00:00<?, ?it/s]